In [1]:
path = "https://raw.githubusercontent.com/shobhit-nigam/knowledgeclan/main/datasets/income.csv"

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score

In [3]:
dfa = pd.read_csv(path)

In [4]:
str_cols = ['workclass', 'education','marital_status', 'occupation','relationship','race',
 'sex','native_country','high_income']

In [5]:
for name in str_cols:
    col = pd.Categorical(dfa[name])
    dfa[name] = col.codes

In [6]:
dfa = dfa.sample(frac=1, random_state=1)    #num= len(dfa)

In [7]:
list_cols = ['age', 'workclass', 'education_num', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'hours_per_week', 'native_country']

In [8]:
train_max_row = round(dfa.shape[0] * 0.8)
train = dfa.iloc[:train_max_row]
test = dfa.iloc[train_max_row:]

In [9]:
tree_count = 10
bag_ratio = 6/10  # (60%)
predictions = []

for i in range(tree_count):
    bag = train.sample(frac=bag_ratio, replace=True, random_state=i)
    
    model = DecisionTreeClassifier(min_samples_leaf=2, random_state=1)
    model.fit(bag[list_cols], bag["high_income"])
    
    predictions.append(model.predict_proba(test[list_cols])[:, 1])
    combination = np.round((np.sum(predictions, axis=0))/(len(predictions)))
    
print(roc_auc_score(test["high_income"], combination)) 

0.7325780430117632


In [10]:
#Gini 

predictions = []

for i in range(tree_count):
    bag = train.sample(frac=bag_ratio, replace=True, random_state=i)
    
    model = DecisionTreeClassifier(min_samples_leaf=2, random_state=1, splitter="random", max_features="auto")
    model.fit(bag[list_cols], bag["high_income"])
    
    predictions.append(model.predict_proba(test[list_cols])[:, 1])
    combination = np.round((np.sum(predictions, axis=0))/(len(predictions)))
    
print(roc_auc_score(test["high_income"], combination)) 

0.7306169822846672


In [15]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=150, random_state=1, min_samples_leaf=2)
model.fit(train[list_cols], train["high_income"])

predictions = model.predict(test[list_cols])
print(roc_auc_score(test["high_income"], predictions)) 

0.7383261438215623


In [22]:
model = DecisionTreeClassifier(random_state=1, min_samples_leaf=5)
model.fit(train[list_cols], train["high_income"])

predictions = model.predict(train[list_cols])
print(f"auc for train with DT {roc_auc_score(train['high_income'], predictions)}") 

auc for train with DT 0.8195052883249438


In [23]:
predictions = model.predict(test[list_cols])
print(f"auc for test with DT {roc_auc_score(test['high_income'], predictions)}") 

auc for test with DT 0.7145347179800449


In [24]:
model = RandomForestClassifier(n_estimators=150, random_state=1, min_samples_leaf=5)
model.fit(train[list_cols], train["high_income"])
predictions = model.predict(train[list_cols])
print(f"auc for train with rf {roc_auc_score(train['high_income'], predictions)}") 
 

auc for train with rf 0.7911471422869599


In [25]:

predictions = model.predict(test[list_cols])
print(f"auc for test with rf {roc_auc_score(test['high_income'], predictions)}")

auc for test with rf 0.7491663612298921
